<a href="https://colab.research.google.com/github/yunyeong-choi/2022fin/blob/master/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C%20%EB%B0%B0%EC%9A%B0%EB%8A%94%20%EC%8B%A4%EC%A0%84%20%EA%B8%88%EC%9C%B5%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D(%ED%80%80%ED%8A%B8)/10%EA%B0%95_%EB%93%80%EC%96%BC%EB%AA%A8%EB%A9%98%ED%85%80_%EC%A0%84%EB%9E%B5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

20221003

# 듀얼모멘텀 전략 구현

## 라이브러리 임포트

In [ ]:
# COLAB 환경을 위한 나눔고딕 한글폰트 설치  --> 진행 후,  런타임 > 런타임 다시 시작을 해 주세요. 다시 시작 후에는 이 부분 실행 X
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 0s (19.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [ ]:
# matplotlib 임포트,  주피터 출력설정,  한글 글꼴 지정
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['font.family'] = 'NanumBarunGothic'  # (Colab 용) 한글 출력을 위한 글꼴 설정

In [ ]:
import numpy as np
import pandas as pd

## 데이터 불러오기

In [ ]:
!wget "https://drive.google.com/uc?export=download&id=1VrrOVibE-oxdxY30SnfCDJ6Ghv38jXm1" -O "stock_data.zip"

--2022-10-03 15:22:55--  https://drive.google.com/uc?export=download&id=1VrrOVibE-oxdxY30SnfCDJ6Ghv38jXm1
Resolving drive.google.com (drive.google.com)... 74.125.31.102, 74.125.31.139, 74.125.31.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.31.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-4g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tqa62a2jjb2i7602hjaaip656qbobvqg/1664810550000/03135380322884973343/*/1VrrOVibE-oxdxY30SnfCDJ6Ghv38jXm1?e=download&uuid=570193b5-faaa-485f-9217-1772fc2356fa [following]
--2022-10-03 15:22:56--  https://doc-00-4g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tqa62a2jjb2i7602hjaaip656qbobvqg/1664810550000/03135380322884973343/*/1VrrOVibE-oxdxY30SnfCDJ6Ghv38jXm1?e=download&uuid=570193b5-faaa-485f-9217-1772fc2356fa
Resolving doc-00-4g-docs.googleusercontent.com (doc-00-4g-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b

In [ ]:
!unzip stock_data.zip

Archive:  stock_data.zip
  inflating: stock.adj_close.csv     
  inflating: stock.eq.csv            
  inflating: stock.liab.csv          
  inflating: stock.mc.csv            
  inflating: stock.ni.csv            
  inflating: stock.op.csv            
  inflating: stock.sales.csv         
  inflating: stockinfo.itemname.csv  
  inflating: stockinfo.sector.csv    


## 3.2. 주식 데이터 및 기업 분석 배우기  
### [ 데이터 설명 ]
**일별 데이터:**
- 데이터 파일: __stock.*{테이블이름}*.csv__
- 데이터 기간: 2018-06-01 ~ 2020-10-14, 약 2년 4개월, 비영업일 제외
- 종목 수: 2,278 개
- 단위 (테이블이름, 설명)
    - 원 단위: adj_close 주가
    - 백만원 단위 : mc 시가총액, sales 매출액, op 영업이익, ni 순이익, liab 부채, eq 자본, asset_cur 유동자산 
        
  
**색인 데이터:**
- __stockinfo.itemname.csv__ : 종목 코드와 종목 명
- __stockinfo.sector.csv__ : 종목 코드와 해당하는 섹터

#### 일자별 주식 데이터 불러오기 (dict 및 for문 사용)

In [ ]:
d = {}

# For문 이용하여 읽기- 주가, 시가총액, 매출액, 영업익, 순이익, 부채, 자본
# adj_close : 주가
# mc : 시가총액
# sales : 매출액
# op : 영업익
# ni : 순이익
# liab : 부채
# eq : 자본
for data_name in ['adj_close', 'mc', 'sales', 'op', 'ni', 'liab', 'eq']:
    print(f"read csv {data_name} ...")
    d[data_name] = pd.read_csv(f"stock.{data_name}.csv", index_col=0, encoding='cp949', parse_dates=True)
    print(f"done!  {d[data_name].shape}")

read csv adj_close ...
done!  (2278, 582)
read csv mc ...
done!  (2278, 582)
read csv sales ...
done!  (2278, 582)
read csv op ...
done!  (2278, 582)
read csv ni ...
done!  (2278, 582)
read csv liab ...
done!  (2278, 582)
read csv eq ...
done!  (2278, 582)


In [ ]:
# 데이터 확인하기
d

{'adj_close':          2018-06-01  2018-06-04  2018-06-05  2018-06-07  2018-06-08  \
 Symbol                                                                
 A000020     11550.0     11750.0     11700.0     11650.0     11500.0   
 A000030     15500.0     15950.0     16050.0     16500.0     16600.0   
 A000040      2992.0      3021.0      3025.0      3069.0      3045.0   
 A000050     13200.0     13550.0     13600.0     13800.0     13800.0   
 A000060     20050.0     20050.0     20150.0     20050.0     20400.0   
 ...             ...         ...         ...         ...         ...   
 A950160     38600.0     36400.0     36350.0     38700.0     40200.0   
 A950170     13850.0     14250.0     14450.0     14300.0     14150.0   
 A950180         NaN         NaN         NaN         NaN         NaN   
 A950190         NaN         NaN         NaN         NaN         NaN   
 A950200         NaN         NaN         NaN         NaN         NaN   
 
          2018-06-11  2018-06-12  2018-06-14  201

In [ ]:
d['adj_close']

,2018-06-01,2018-06-04,2018-06-05,2018-06-07,2018-06-08,2018-06-11,2018-06-12,2018-06-14,2018-06-15,2018-06-18,...,2020-09-25,2020-09-28,2020-09-29,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-12,2020-10-13,2020-10-14
Symbol,,,,,,,,,,,,,,,,,,,,,
A000020,11550.0,11750.0,11700.0,11650.0,11500.0,11500.0,11650.0,12000.0,11800.0,11450.0,...,22100.0,22000.0,21850.0,24100.0,23750.0,23500.0,24150.0,24000.0,24300.0,23850.0
A000030,15500.0,15950.0,16050.0,16500.0,16600.0,16650.0,16850.0,16500.0,16100.0,16350.0,...,14800.0,14800.0,14800.0,14800.0,14800.0,14800.0,14800.0,14800.0,14800.0,14800.0
A000040,2992.0,3021.0,3025.0,3069.0,3045.0,2984.0,2976.0,3033.0,3033.0,3017.0,...,771.0,743.0,767.0,829.0,810.0,860.0,863.0,863.0,850.0,874.0
A000050,13200.0,13550.0,13600.0,13800.0,13800.0,13800.0,13650.0,13850.0,14500.0,14550.0,...,10750.0,10500.0,10550.0,10850.0,10900.0,10900.0,11000.0,11000.0,10850.0,10800.0
A000060,20050.0,20050.0,20150.0,20050.0,20400.0,20150.0,20700.0,20950.0,20150.0,20900.0,...,12750.0,12750.0,12850.0,13150.0,13300.0,13350.0,13900.0,13950.0,13700.0,13550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A950160,38600.0,36400.0,36350.0,38700.0,40200.0,39250.0,38300.0,38350.0,39000.0,37600.0,...,8010.0,8010.0,8010.0,8010.0,8010.0,8010.0,8010.0,8010.0,8010.0,8010.0
A950170,13850.0,14250.0,14450.0,14300.0,14150.0,13650.0,14050.0,13600.0,13550.0,11900.0,...,3820.0,3900.0,4050.0,4070.0,4190.0,4160.0,4320.0,4445.0,4440.0,4335.0
A950180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12400.0,12800.0,12900.0,12800.0,12850.0,13050.0,13050.0,12900.0,12650.0,13350.0


In [ ]:
d['mc']

,2018-06-01,2018-06-04,2018-06-05,2018-06-07,2018-06-08,2018-06-11,2018-06-12,2018-06-14,2018-06-15,2018-06-18,...,2020-09-25,2020-09-28,2020-09-29,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-12,2020-10-13,2020-10-14
Symbol,,,,,,,,,,,,,,,,,,,,,
A000020,322608.0,328195.0,326798.0,325402.0,321212.0,321212.0,325402.0,335178.0,329591.0,319815.0,...,617300.0,614500.0,610300.0,673100.0,663400.0,656400.0,674500.0,670400.0,678700.0,666200.0
A000030,10478000.0,10782200.0,10849800.0,11154000.0,11221600.0,11255400.0,11390600.0,11154000.0,10883600.0,11052600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A000040,88263.0,89101.0,89221.0,90538.0,89820.0,88023.0,87784.0,89460.0,89460.0,88981.0,...,70700.0,68100.0,70300.0,76000.0,74200.0,78800.0,79100.0,79100.0,77900.0,80100.0
A000050,361882.0,371477.0,372848.0,378331.0,378331.0,378331.0,374218.0,379701.0,397521.0,398892.0,...,294700.0,287900.0,289200.0,297500.0,298800.0,298800.0,301600.0,301600.0,297500.0,296100.0
A000060,2212277.0,2212277.0,2223311.0,2212277.0,2250895.0,2223311.0,2283997.0,2311581.0,2223311.0,2306064.0,...,1449400.0,1449400.0,1460800.0,1494900.0,1511900.0,1517600.0,1580200.0,1585800.0,1557400.0,1540400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A950160,2350704.0,2216726.0,2213681.0,2356794.0,2448142.0,2390288.0,2332434.0,2335479.0,2375063.0,2291614.0,...,489600.0,489600.0,489600.0,489600.0,489600.0,489600.0,489600.0,489600.0,489600.0,489600.0
A950170,484826.0,498829.0,505830.0,500579.0,495328.0,477825.0,491828.0,476075.0,474325.0,416566.0,...,133700.0,136500.0,141800.0,142500.0,146700.0,145600.0,151200.0,155600.0,155400.0,151700.0
A950180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,261200.0,269600.0,271700.0,269600.0,270600.0,274900.0,274900.0,271700.0,266400.0,281200.0


In [ ]:
d['sales']

,2018-06-01,2018-06-04,2018-06-05,2018-06-07,2018-06-08,2018-06-11,2018-06-12,2018-06-14,2018-06-15,2018-06-18,...,2020-09-25,2020-09-28,2020-09-29,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-12,2020-10-13,2020-10-14
Symbol,,,,,,,,,,,,,,,,,,,,,
A000020,2.752047e+05,2.752047e+05,2.752047e+05,2.752047e+05,2.752047e+05,2.752047e+05,2.752047e+05,2.752047e+05,2.752047e+05,2.752047e+05,...,2.943950e+05,2.943950e+05,2.943950e+05,2.943950e+05,2.943950e+05,2.943950e+05,2.943950e+05,2.943950e+05,2.943950e+05,2.943950e+05
A000030,8.741658e+06,8.741658e+06,8.741658e+06,8.741658e+06,8.741658e+06,8.741658e+06,8.741658e+06,8.741658e+06,8.741658e+06,8.741658e+06,...,9.293464e+06,9.293464e+06,9.293464e+06,9.293464e+06,9.293464e+06,9.293464e+06,9.293464e+06,9.293464e+06,9.293464e+06,9.293464e+06
A000040,3.535273e+04,3.535273e+04,3.535273e+04,3.535273e+04,3.535273e+04,3.535273e+04,3.535273e+04,3.535273e+04,3.535273e+04,3.535273e+04,...,1.304825e+05,1.304825e+05,1.304825e+05,1.304825e+05,1.304825e+05,1.304825e+05,1.304825e+05,1.304825e+05,1.304825e+05,1.304825e+05
A000050,3.506402e+05,3.506402e+05,3.506402e+05,3.506402e+05,3.506402e+05,3.506402e+05,3.506402e+05,3.506402e+05,3.506402e+05,3.506402e+05,...,3.328343e+05,3.328343e+05,3.328343e+05,3.328343e+05,3.328343e+05,3.328343e+05,3.328343e+05,3.328343e+05,3.328343e+05,3.328343e+05
A000060,6.576727e+06,6.576727e+06,6.576727e+06,6.576727e+06,6.576727e+06,6.576727e+06,6.576727e+06,6.576727e+06,6.576727e+06,6.576727e+06,...,8.399882e+06,8.399882e+06,8.399882e+06,8.399882e+06,8.399882e+06,8.399882e+06,8.399882e+06,8.399882e+06,8.399882e+06,8.399882e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A950160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A950170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A950180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
d['ni']

,2018-06-01,2018-06-04,2018-06-05,2018-06-07,2018-06-08,2018-06-11,2018-06-12,2018-06-14,2018-06-15,2018-06-18,...,2020-09-25,2020-09-28,2020-09-29,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-12,2020-10-13,2020-10-14
Symbol,,,,,,,,,,,,,,,,,,,,,
A000020,4.829017e+04,4.829017e+04,4.829017e+04,4.829017e+04,4.829017e+04,4.829017e+04,4.829017e+04,4.829017e+04,4.829017e+04,4.829017e+04,...,1.716170e+04,1.716170e+04,1.716170e+04,1.716170e+04,1.716170e+04,1.716170e+04,1.716170e+04,1.716170e+04,1.716170e+04,1.716170e+04
A000030,1.481877e+06,1.481877e+06,1.481877e+06,1.481877e+06,1.481877e+06,1.481877e+06,1.481877e+06,1.481877e+06,1.481877e+06,1.481877e+06,...,2.058499e+06,2.058499e+06,2.058499e+06,2.058499e+06,2.058499e+06,2.058499e+06,2.058499e+06,2.058499e+06,2.058499e+06,2.058499e+06
A000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A000050,2.437947e+04,2.437947e+04,2.437947e+04,2.437947e+04,2.437947e+04,2.437947e+04,2.437947e+04,2.437947e+04,2.437947e+04,2.437947e+04,...,4.046476e+04,4.046476e+04,4.046476e+04,4.046476e+04,4.046476e+04,4.046476e+04,4.046476e+04,4.046476e+04,4.046476e+04,4.046476e+04
A000060,3.402020e+05,3.402020e+05,3.402020e+05,3.402020e+05,3.402020e+05,3.402020e+05,3.402020e+05,3.402020e+05,3.402020e+05,3.402020e+05,...,3.524787e+05,3.524787e+05,3.524787e+05,3.524787e+05,3.524787e+05,3.524787e+05,3.524787e+05,3.524787e+05,3.524787e+05,3.524787e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A950160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A950170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A950180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 색인 데이터 로드 및 병합 (concat 사용)

In [ ]:
itemname = pd.read_csv("stockinfo.itemname.csv", index_col=0, encoding='cp949')
sector = pd.read_csv("stockinfo.sector.csv", index_col=0, encoding='cp949')

In [ ]:
stock_info = pd.concat([itemname, sector], axis=1)

In [ ]:
stock_info

,itemname,Sector
Symbol,,
A000020,동화약품,제약_및_바이오
A000030,우리은행,NaN
A000040,KR모터스,자동차_및_부품
A000050,경방,내구_소비재_및_의류
A000060,메리츠화재,보험
...,...,...
A950160,코오롱티슈진,NaN
A950170,JTC,NaN
A950180,SNK,NaN


##### 여러 투자 기간 (1일, 3일, ... 60일, 120일) 의 수익률을 계산하여, 딕셔너리 d에 할당하기
* `.pct_change()`는 비율을 계산하므로, 백분율 % 로 만들기 위해 100을 곱해줍니다.

In [ ]:
for hold_days in [1, 3, 5, 10, 20, 60, 120]:
    table_name = f'return_{hold_days}d'
    d[table_name] = d['adj_close'].pct_change(periods = hold_days, axis=1) * 100
    
    print(f"투자기간 {hold_days} 일, 테이블 {table_name} 연산 완료 !")

투자기간 1 일, 테이블 return_1d 연산 완료 !
투자기간 3 일, 테이블 return_3d 연산 완료 !
투자기간 5 일, 테이블 return_5d 연산 완료 !
투자기간 10 일, 테이블 return_10d 연산 완료 !
투자기간 20 일, 테이블 return_20d 연산 완료 !
투자기간 60 일, 테이블 return_60d 연산 완료 !
투자기간 120 일, 테이블 return_120d 연산 완료 !


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 딕셔너리d에 잘 들어갔나 확인하기
d['return_1d']

,2018-06-01,2018-06-04,2018-06-05,2018-06-07,2018-06-08,2018-06-11,2018-06-12,2018-06-14,2018-06-15,2018-06-18,...,2020-09-25,2020-09-28,2020-09-29,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-12,2020-10-13,2020-10-14
Symbol,,,,,,,,,,,,,,,,,,,,,
A000020,NaN,1.731602,-0.425532,-0.427350,-1.287554,0.000000,1.304348,3.004292,-1.666667,-2.966102,...,4.739336,-0.452489,-0.681818,10.297483,-1.452282,-1.052632,2.765957,-0.621118,1.250000,-1.851852
A000030,NaN,2.903226,0.626959,2.803738,0.606061,0.301205,1.201201,-2.077151,-2.424242,1.552795,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A000040,NaN,0.969251,0.132406,1.454545,-0.782014,-2.003284,-0.268097,1.915323,0.000000,-0.527530,...,-0.772201,-3.631647,3.230148,8.083442,-2.291918,6.172840,0.348837,0.000000,-1.506373,2.823529
A000050,NaN,2.651515,0.369004,1.470588,0.000000,0.000000,-1.086957,1.465201,4.693141,0.344828,...,-7.327586,-2.325581,0.476190,2.843602,0.460829,0.000000,0.917431,0.000000,-1.363636,-0.460829
A000060,NaN,0.000000,0.498753,-0.496278,1.745636,-1.225490,2.729529,1.207729,-3.818616,3.722084,...,0.000000,0.000000,0.784314,2.334630,1.140684,0.375940,4.119850,0.359712,-1.792115,-1.094891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A950160,NaN,-5.699482,-0.137363,6.464924,3.875969,-2.363184,-2.420382,0.130548,1.694915,-3.589744,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A950170,NaN,2.888087,1.403509,-1.038062,-1.048951,-3.533569,2.930403,-3.202847,-0.367647,-12.177122,...,0.526316,2.094241,3.846154,0.493827,2.948403,-0.715990,3.846154,2.893519,-0.112486,-2.364865
A950180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.401606,3.225806,0.781250,-0.775194,0.390625,1.556420,0.000000,-1.149425,-1.937984,5.533597


In [ ]:
# 딕셔너리d에 잘 들어갔나 확인하기
d['return_5d']

,2018-06-01,2018-06-04,2018-06-05,2018-06-07,2018-06-08,2018-06-11,2018-06-12,2018-06-14,2018-06-15,2018-06-18,...,2020-09-25,2020-09-28,2020-09-29,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-12,2020-10-13,2020-10-14
Symbol,,,,,,,,,,,,,,,,,,,,,
A000020,NaN,NaN,NaN,NaN,NaN,-0.432900,-0.851064,2.564103,1.287554,-0.434783,...,-11.776447,-12.175649,-9.147609,4.103672,12.559242,6.334842,9.772727,9.839817,0.829876,0.421053
A000030,NaN,NaN,NaN,NaN,NaN,7.419355,5.642633,2.803738,-2.424242,-1.506024,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A000040,NaN,NaN,NaN,NaN,NaN,-0.267380,-1.489573,0.264463,-1.173021,-0.919540,...,-3.140704,-4.129032,5.212620,16.596343,4.247104,11.543450,16.150740,12.516297,2.533172,7.901235
A000050,NaN,NaN,NaN,NaN,NaN,4.545455,0.738007,1.838235,5.072464,5.434783,...,0.000000,-2.325581,1.442308,4.830918,-6.034483,1.395349,4.761905,4.265403,0.000000,-0.917431
A000060,NaN,NaN,NaN,NaN,NaN,0.498753,3.241895,3.970223,0.498753,2.450980,...,-3.409091,-2.298851,1.181102,1.544402,4.313725,4.705882,9.019608,8.560311,4.182510,1.879699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A950160,NaN,NaN,NaN,NaN,NaN,1.683938,5.219780,5.502063,0.775194,-6.467662,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A950170,NaN,NaN,NaN,NaN,NaN,-1.444043,-1.403509,-5.882353,-5.244755,-15.901060,...,-5.795314,-1.515152,4.786546,4.358974,10.263158,8.900524,10.769231,9.753086,9.090909,3.460621
A950180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.806691,-2.290076,1.176471,1.185771,3.212851,5.241935,1.953125,0.000000,-1.171875,3.891051


#### DataFrame을 입력하면, stock_info를 앞에 붙여주는 함수 `get_df_with_info()` 만들기

In [ ]:
def get_df_with_info(data_df):
    return pd.concat([stock_info, data_df], axis=1)

In [ ]:
d['adj_close'].iloc[:,-5:]

,2020-10-07,2020-10-08,2020-10-12,2020-10-13,2020-10-14
Symbol,,,,,
A000020,23500.0,24150.0,24000.0,24300.0,23850.0
A000030,14800.0,14800.0,14800.0,14800.0,14800.0
A000040,860.0,863.0,863.0,850.0,874.0
A000050,10900.0,11000.0,11000.0,10850.0,10800.0
A000060,13350.0,13900.0,13950.0,13700.0,13550.0
...,...,...,...,...,...
A950160,8010.0,8010.0,8010.0,8010.0,8010.0
A950170,4160.0,4320.0,4445.0,4440.0,4335.0
A950180,13050.0,13050.0,12900.0,12650.0,13350.0


In [ ]:
get_df_with_info(d['adj_close'].iloc[:,-5:])

,itemname,Sector,2020-10-07,2020-10-08,2020-10-12,2020-10-13,2020-10-14
Symbol,,,,,,,
A000020,동화약품,제약_및_바이오,23500.0,24150.0,24000.0,24300.0,23850.0
A000030,우리은행,NaN,14800.0,14800.0,14800.0,14800.0,14800.0
A000040,KR모터스,자동차_및_부품,860.0,863.0,863.0,850.0,874.0
A000050,경방,내구_소비재_및_의류,10900.0,11000.0,11000.0,10850.0,10800.0
A000060,메리츠화재,보험,13350.0,13900.0,13950.0,13700.0,13550.0
...,...,...,...,...,...,...,...
A950160,코오롱티슈진,NaN,8010.0,8010.0,8010.0,8010.0,8010.0
A950170,JTC,NaN,4160.0,4320.0,4445.0,4440.0,4335.0
A950180,SNK,NaN,13050.0,13050.0,12900.0,12650.0,13350.0


## 가설과 실험 수행하기

In [ ]:
# 전체 날짜 리스트 만들어두기
date_list = d['adj_close'].columns
date_list

Index(['2018-06-01', '2018-06-04', '2018-06-05', '2018-06-07', '2018-06-08',
       '2018-06-11', '2018-06-12', '2018-06-14', '2018-06-15', '2018-06-18',
       ...
       '2020-09-25', '2020-09-28', '2020-09-29', '2020-10-05', '2020-10-06',
       '2020-10-07', '2020-10-08', '2020-10-12', '2020-10-13', '2020-10-14'],
      dtype='object', length=582)

#### 주식 데이터 분석3. 듀얼 모멘텀 전략 구현(3개월 기준)
##### **상대모멘텀 : 3개월 기준 수익률 상위 10개 종목**
##### **절대모멘텀 : 3개월 기준 수익률 > 0**

In [ ]:
# 60일간의 투자 수익률을 계산할 예정이므로, 뒤에서 60번째 날짜를 골라야 합니다.
date_list[-61]

'2020-07-15'

In [ ]:
# 매수할 날짜 지정하기
buy_date = '2020-07-15'

In [ ]:
# 60일 (달력 기준 3달) 투자기간을 기준으로, 수익률 내림차순 정렬하기
d['return_60d'].loc[:, buy_date].sort_values(ascending=False).iloc[:10]

Symbol
A205470    431.728665
A336260    383.606557
A019170    285.230769
A101360    282.273763
A036810    260.354223
A003680    253.757225
A153710    211.859444
A196170    211.472466
A057680    197.744361
A025900    193.023256
Name: 2020-07-15, dtype: float64

In [ ]:
# 선별된 종목의 인덱스 뽑아내기
buy_stock_index = d['return_60d'].loc[:, buy_date].sort_values(ascending=False).iloc[:10].index
buy_stock_index

Index(['A205470', 'A336260', 'A019170', 'A101360', 'A036810', 'A003680',
       'A153710', 'A196170', 'A057680', 'A025900'],
      dtype='object', name='Symbol')

In [ ]:
# 구매 시점의 전 종목의 평균 수익률 (절대모멘텀)
d['return_20d'].loc[:, buy_date].mean()

5.825519077126442

#### 시뮬레이션1. 3개월 모멘텀을 통해 선별한 종목에 투자하면 수익률이 어떨까?

In [ ]:
# 전 종목의 평균 수익률
d['return_60d'].loc[:, '2020-10-14'].mean()

13.82906109580941

In [ ]:
# 선별한 종목의 평균 수익률
d['return_60d'].loc[buy_stock_index, '2020-10-14'].mean()

15.820618521022215

#### 주식 데이터 분석3. 듀얼 모멘텀 전략 구현(6개월 기준)
##### **상대모멘텀 : 6개월 기준 수익률 상위 10개 종목**
##### **절대모멘텀 : 6개월 기준 수익률 > 0**

In [ ]:
# 120일간의 투자 수익률을 계산할 예정이므로, 뒤에서 120번째 날짜를 골라야 합니다.
date_list[-121]

'2020-04-17'

In [ ]:
# 매수할 날짜 지정하기
buy_date = '2020-04-17'

In [ ]:
# 120일 (달력 기준 6달) 투자기간을 기준으로, 수익률 내림차순 정렬하기
d['return_120d'].loc[:, buy_date].sort_values(ascending=False).iloc[:10]

Symbol
A058110    335.013263
A011000    277.777778
A096530    268.374165
A180640    259.016393
A078590    224.691358
A084650    184.710018
A290650    181.720430
A253840    180.000000
A256940    173.326316
A019170    162.520194
Name: 2020-04-17, dtype: float64

In [ ]:
# 선별된 종목의 인덱스 뽑아내기
buy_stock_index = d['return_120d'].loc[:, buy_date].sort_values(ascending=False).iloc[:10].index
buy_stock_index

Index(['A058110', 'A011000', 'A096530', 'A180640', 'A078590', 'A084650',
       'A290650', 'A253840', 'A256940', 'A019170'],
      dtype='object', name='Symbol')

In [ ]:
# 구매 시점의 전 종목의 평균 수익률 (절대모멘텀)
d['return_120d'].loc[:, buy_date].mean()
# 절대 모멘텀이 -이므로 투자를 진행하지 않음

-7.14181318937702

##### 절대 모멘텀이 -이므로 투자를 진행하지 않음

#### 주식 데이터 분석3. 듀얼 모멘텀 전략 구현(1개월 기준)
##### **상대모멘텀 : 1개월 기준 수익률 상위 10개 종목**
##### **절대모멘텀 : 1개월 기준 수익률 > 0**

In [ ]:
# 20일간의 투자 수익률을 계산할 예정이므로, 뒤에서 20번째 날짜를 골라야 합니다.
date_list[-21]

'2020-09-10'

In [ ]:
# 매수할 날짜 지정하기
buy_date = '2020-09-10'

In [ ]:
# 120일 (달력 기준 6달) 투자기간을 기준으로, 수익률 내림차순 정렬하기
d['return_20d'].loc[:, buy_date].sort_values(ascending=False).iloc[:10]

Symbol
A057880    343.223443
A033270    283.877159
A269620    268.127490
A047920    241.197822
A018000    228.275862
A011000    211.387900
A082850    197.107438
A029480    169.965278
A051980    154.929577
A217330    148.706897
Name: 2020-09-10, dtype: float64

In [ ]:
# 선별된 종목의 인덱스 뽑아내기
buy_stock_index = d['return_20d'].loc[:, buy_date].sort_values(ascending=False).iloc[:10].index
buy_stock_index

Index(['A057880', 'A033270', 'A269620', 'A047920', 'A018000', 'A011000',
       'A082850', 'A029480', 'A051980', 'A217330'],
      dtype='object', name='Symbol')

In [ ]:
# 구매 시점의 전 종목의 평균 수익률 (절대모멘텀)
d['return_20d'].loc[:, buy_date].mean()

4.9323767353942625

In [ ]:
# 전 종목의 평균 수익률
d['return_20d'].loc[:, '2020-10-14'].mean()

1.8118556177803595

In [ ]:
# 선별한 종목의 평균 수익률
d['return_20d'].loc[buy_stock_index, '2020-10-14'].mean()

-5.0518912343476945